In [2]:
from pathlib import Path
import pandas as pd
# import h3
import numpy as np
from tqdm import tqdm
import json
import joblib
from typing import List
from xgboost import XGBClassifier
from statistics import mean
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

In [3]:
# метрика контеста
def mean_binary_cross_entropy(predictuion, target):
    eps = 1e-8
    mbce = (-np.log(np.clip(predictuion, eps, 1 - eps)) * target \
           - np.log(np.clip(1 - predictuion, eps, 1 - eps)) * (1 - target)).sum(axis=1).mean()
    return mbce

In [4]:
!pip install osmnx networkx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 2.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: shapely
    Found existing installation: Shapely 1.8.5.post1
    Uninstalling Shapely-1.8.5.post1:
      Successfully uninstalled Shapely-1.8.5.post1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.


In [5]:
!pip uninstall -y osmnx shapely
!pip install osmnx==1.1.1 shapely==1.8.5.post1

Found existing installation: osmnx 1.9.4
Uninstalling osmnx-1.9.4:
  Successfully uninstalled osmnx-1.9.4
Found existing installation: shapely 2.0.6
Uninstalling shapely-2.0.6:
  Successfully uninstalled shapely-2.0.6
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.7 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.


In [ ]:
!conda install -c conda-forge osmnx

Retrieving notices: ...working... done
Channels:
 - conda-forge
 - rapidsai
 - nvidia
 - nodefaults
 - defaults
Platform: linux-64
Solving environment: \ 

In [ ]:
import osmnx as ox
import networkx as nx

# Координаты начальной и конечной точек
origin = (37.7749, -122.4194)  # Пример: Сан-Франциско
destination = (34.0522, -118.2437)  # Пример: Лос-Анджелес

# Загрузка графа дорожной сети
G = ox.graph_from_point(origin, dist=50000, network_type="drive")

# Получение ближайших узлов к координатам
orig_node = ox.distance.nearest_nodes(G, origin[1], origin[0])
dest_node = ox.distance.nearest_nodes(G, destination[1], destination[0])

# Поиск кратчайшего маршрута
route = nx.shortest_path(G, orig_node, dest_node, weight="length")
distance = nx.shortest_path_length(G, orig_node, dest_node, weight="length")

print("Расстояние по дороге:", distance / 1000, "км")

In [42]:
moscow_df

,id,tags,lat,lon
0,1000,"[(comment, ООО ""АСП-ГРУПП"", ИНН 7735108290, до...",55.995115,37.207532
1,26609007,"[(ele, 190), (iata, SVO), (icao, UUEE), (name,...",55.987408,37.420944
2,26999673,"[(alt_name, Москва-Октябрьская), (esr:user, 06...",55.778834,37.653721
3,26999677,None,55.784881,37.649458
4,26999678,None,55.785265,37.649081
...,...,...,...,...
8867808,11559263044,None,55.536454,36.987296
8867809,11559263045,None,55.536431,36.987332
8867810,11559263046,None,55.531485,36.999239
8867811,11559263047,None,55.532101,36.999169


In [1]:
import pandas as pd

# Загрузка данных локаций и транзакций
transactions_df = pd.read_parquet("/kaggle/input/geo-analize-bankomat/transactions.parquet")
target_df = pd.read_parquet("/kaggle/input/geo-analize-bankomat/target.parquet")
moscow_df = pd.read_parquet("/kaggle/input/geo-analize-bankomat/moscow.parquet")

# Загрузка h3 индексов
with open("/kaggle/input/geo-analize-bankomat/hexses_data.lst", "r") as f:
    hexses_data = [x.strip() for x in f.readlines()]

with open("/kaggle/input/geo-analize-bankomat/hexses_target.lst", "r") as f:
    hexses_target = [x.strip() for x in f.readlines()]

# Преобразуем hexses_data и hexses_target в DataFrame
hexses_data_df = pd.DataFrame(hexses_data, columns=["h3_09"])
hexses_target_df = pd.DataFrame(hexses_target, columns=["h3_09"])

# # Фильтрация данных в moscow_df на основе локаций hexses_data и hexses_target
# locations_df = pd.concat([hexses_data_df, hexses_target_df]).drop_duplicates()
# moscow_filtered_df = moscow_df[moscow_df["h3_09"].isin(locations_df["h3_09"])]

# # Объединение с транзакциями
# transactions_with_coords = transactions_df.merge(moscow_filtered_df, on="h3_09", how="left")
# target_with_coords = target_df.merge(moscow_filtered_df, on="h3_09", how="left")

# # Теперь transactions_with_coords и target_with_coords содержат координаты для локаций

In [ ]:
data_coordinates = []
for location in tqdm(hexses_data):
    lat, lng = h3.h3_to_geo(location)
    data_coordinates.append([location,lat, lng])
coordinates_df = pd.DataFrame(data_coordinates,columns=['h3_09','lat','lng'])

In [37]:
final_df

,h3_09,customer_id,datetime_id,count,sum,avg,min,max,std,count_distinct,mcc_code
0,8911aa4c62fffff,1,3,1,3346.65,3346.650,3346.65,3346.65,NaN,1,13
1,8911aa7b5b3ffff,4,3,1,450.00,450.000,450.00,450.00,NaN,1,8
2,8911aa63623ffff,5,3,10,11035.69,1103.569,59.00,3620.18,1190.530333,6,13
3,8911aa48577ffff,9,2,2,628.00,314.000,295.00,333.00,26.870058,2,5
4,8911aa78297ffff,11,2,1,4155.00,4155.000,4155.00,4155.00,NaN,1,10
...,...,...,...,...,...,...,...,...,...,...,...
4151091,8911aa6216bffff,50600,2,1,4820.00,4820.000,4820.00,4820.00,NaN,1,0
4151092,891181b6423ffff,23168,1,1,70.00,70.000,70.00,70.00,NaN,1,13
4151093,8911aa7ab83ffff,62835,1,1,400.00,400.000,400.00,400.00,NaN,1,8
4151094,8911aa01dc7ffff,71229,1,1,6797.00,6797.000,6797.00,6797.00,NaN,1,5


In [32]:
loc_out = [x for x in hexses_target if x not in hexses_data]
loc_out

,h3_09,customer_id,datetime_id,count,sum,avg,min,max,std,count_distinct,mcc_code
0,8911aa4c62fffff,1,3,1,3346.65,3346.650,3346.65,3346.65,NaN,1,13
1,8911aa7b5b3ffff,4,3,1,450.00,450.000,450.00,450.00,NaN,1,8
2,8911aa63623ffff,5,3,10,11035.69,1103.569,59.00,3620.18,1190.530333,6,13
3,8911aa48577ffff,9,2,2,628.00,314.000,295.00,333.00,26.870058,2,5
4,8911aa78297ffff,11,2,1,4155.00,4155.000,4155.00,4155.00,NaN,1,10
...,...,...,...,...,...,...,...,...,...,...,...
4151091,8911aa6216bffff,50600,2,1,4820.00,4820.000,4820.00,4820.00,NaN,1,0
4151092,891181b6423ffff,23168,1,1,70.00,70.000,70.00,70.00,NaN,1,13
4151093,8911aa7ab83ffff,62835,1,1,400.00,400.000,400.00,400.00,NaN,1,8
4151094,8911aa01dc7ffff,71229,1,1,6797.00,6797.000,6797.00,6797.00,NaN,1,5


In [7]:
transactions = pd.read_parquet('/kaggle/input/geo-analize-bankomat/transactions.parquet')
transactions

,h3_09,customer_id,datetime_id,count,sum,avg,min,max,std,count_distinct,mcc_code
0,8911aa4c62fffff,1,3,1,3346.65,3346.650,3346.65,3346.65,NaN,1,13
1,8911aa7b5b3ffff,4,3,1,450.00,450.000,450.00,450.00,NaN,1,8
2,8911aa63623ffff,5,3,10,11035.69,1103.569,59.00,3620.18,1190.530333,6,13
3,8911aa48577ffff,9,2,2,628.00,314.000,295.00,333.00,26.870058,2,5
4,8911aa78297ffff,11,2,1,4155.00,4155.000,4155.00,4155.00,NaN,1,10
...,...,...,...,...,...,...,...,...,...,...,...
4151091,8911aa6216bffff,50600,2,1,4820.00,4820.000,4820.00,4820.00,NaN,1,0
4151092,891181b6423ffff,23168,1,1,70.00,70.000,70.00,70.00,NaN,1,13
4151093,8911aa7ab83ffff,62835,1,1,400.00,400.000,400.00,400.00,NaN,1,8
4151094,8911aa01dc7ffff,71229,1,1,6797.00,6797.000,6797.00,6797.00,NaN,1,5


In [4]:
pip install h3 --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 11.7 MB/s eta 0:00:0000:010:01
Note: you may need to restart the kernel to use updated packages.


In [18]:
hexses_target_path = '/kaggle/input/geo-analize-bankomat/hexses_target.lst'
hexses_data_path = '/kaggle/input/geo-analize-bankomat/hexses_data.lst'
train_data_fn = '/kaggle/input/geo-analize-bankomat/transactions.parquet'

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm

# Загрузка данных из файлов
with open(hexses_target_path, "r") as f:
    hexses_target = [x.strip() for x in f.readlines()]
with open(hexses_data_path, "r") as f:
    hexses_data = [x.strip() for x in f.readlines()]

# Загрузка данных транзакций
transactions = pd.read_parquet(train_data_fn)

# Преобразуем данные локаций в DataFrame
# Предположим, что hexses_data - это список строк, где каждая строка в формате "lat,lon"
data_coordinates = []
for location in tqdm(hexses_data):
    # Разделяем строку на широту и долготу
    lat, lng = map(float, location.split(','))
    data_coordinates.append([location, lat, lng])

# Создание DataFrame из локаций с координатами
coordinates_df = pd.DataFrame(data_coordinates, columns=['location', 'lat', 'lng'])

# Преобразование в GeoDataFrame с точками
coordinates_gdf = gpd.GeoDataFrame(
    coordinates_df, 
    geometry=gpd.points_from_xy(coordinates_df.lng, coordinates_df.lat),
    crs="EPSG:4326"  # Устанавливаем систему координат WGS 84
)

# Если в transactions есть координаты, также преобразуем в GeoDataFrame
transactions_gdf = gpd.GeoDataFrame(
    transactions,
    geometry=gpd.points_from_xy(transactions['lng'], transactions['lat']),
    crs="EPSG:4326"
)

# Объединение данных по пространственному отношению
merged_df = gpd.sjoin(transactions_gdf, coordinates_gdf, how="left", op="intersects")

# Просмотр результата объединения
print(merged_df.head())


  0%|          | 0/8154 [00:00<?, ?it/s]


ValueError: could not convert string to float: '89118180927ffff'

In [17]:
hexses_data

['89118180927ffff',
 '89118180d27ffff',
 '891181820abffff',
 '891181840a7ffff',
 '891181844c3ffff',
 '89118184c93ffff',
 '891181854b7ffff',
 '89118186067ffff',
 '8911818610bffff',
 '89118186173ffff',
 '8911818620bffff',
 '89118186227ffff',
 '89118186233ffff',
 '89118186343ffff',
 '89118186347ffff',
 '8911818635bffff',
 '89118186493ffff',
 '8911818654fffff',
 '89118186813ffff',
 '89118186817ffff',
 '89118186857ffff',
 '89118186883ffff',
 '89118186887ffff',
 '89118186897ffff',
 '8911818689bffff',
 '8911818690bffff',
 '8911818690fffff',
 '8911818692bffff',
 '8911818692fffff',
 '89118186977ffff',
 '89118186ad3ffff',
 '89118186ad7ffff',
 '89118186b13ffff',
 '89118186c13ffff',
 '89118186c23ffff',
 '89118186c37ffff',
 '89118186cc7ffff',
 '89118186cd7ffff',
 '89118186d07ffff',
 '89118186dafffff',
 '89118186e9bffff',
 '89118190177ffff',
 '89118190303ffff',
 '8911819050fffff',
 '891181909afffff',
 '89118190a9bffff',
 '89118190c17ffff',
 '89118190e17ffff',
 '89118190e2fffff',
 '89118190e37ffff',


In [16]:
import pandas as pd

# Загрузка данных транзакций
transactions_df = pd.read_parquet("/kaggle/input/geo-analize-bankomat/transactions.parquet")

# Загрузка данных локаций из hexses_data.lst
with open("/kaggle/input/geo-analize-bankomat/hexses_data.lst") as f:
    hexses_data = f.read().splitlines()

# Преобразование данных локаций в DataFrame
hexses_data_df = pd.DataFrame(hexses_data, columns=["location"])

# Выполняем объединение данных по полю 'location'
merged_df = transactions_df.merge(hexses_data_df, on="location", how="inner")

# Просмотр результата объединения
print(merged_df.head())


KeyError: 'location'

In [10]:
import h3
data_coordinates = []
for location in tqdm(hexses_data):
    lat, lng = h3.h3_to_geo(location)
    lat, lng = h3.h3_to_latlng(location)
    data_coordinates.append([location,lat, lng])
coordinates_df = pd.DataFrame(data_coordinates,columns=['h3_09','lat','lng'])

  0%|          | 0/8154 [00:00<?, ?it/s]


AttributeError: module 'h3' has no attribute 'h3_to_geo'

In [11]:
hexses_target

['8911818610bffff',
 '89118195133ffff',
 '8911819513bffff',
 '891181b2827ffff',
 '891181b2957ffff',
 '891181b2b83ffff',
 '891181b2b97ffff',
 '891181b2ba7ffff',
 '891181b2bb3ffff',
 '891181b2d1bffff',
 '891181b2e6bffff',
 '891181b4513ffff',
 '891181b451bffff',
 '891181b45c3ffff',
 '891181b45d7ffff',
 '891181b4c8fffff',
 '891181b4ca7ffff',
 '891181b4cb3ffff',
 '891181b4cd3ffff',
 '891181b4d2fffff',
 '891181b4d9bffff',
 '891181b4da7ffff',
 '891181b4dbbffff',
 '891181b4dcbffff',
 '891181b4dd3ffff',
 '891181b4dd7ffff',
 '891181b602fffff',
 '891181b6053ffff',
 '891181b6087ffff',
 '891181b608fffff',
 '891181b6093ffff',
 '891181b6097ffff',
 '891181b609bffff',
 '891181b60a7ffff',
 '891181b60bbffff',
 '891181b60d3ffff',
 '891181b6193ffff',
 '891181b6317ffff',
 '891181b6323ffff',
 '891181b634bffff',
 '891181b6357ffff',
 '891181b636bffff',
 '891181b637bffff',
 '891181b638fffff',
 '891181b642fffff',
 '891181b6503ffff',
 '891181b6507ffff',
 '891181b650fffff',
 '891181b6517ffff',
 '891181b6533ffff',
